In [1]:
collections = ['MARKET_INTEL_COLLECTION_US', 'MARKET_INTEL_COLLECTION_EMEA', 'MARKET_INTEL_COLLECTION_LATAM', 'MARKET_INTEL_COLLECTION_APAC']
collection = 'MARKET_INTEL_COLLECTION_APAC'

In [2]:
service_slugs = ['ubereats-marketintel', 'grubhub-marketintel', 'glovo-marketintel','doordash-marketintel','yelp-marketintel']
service_slug = 'ubereats-marketintel'

In [3]:
begin_date = '2022-06-01'
end_date = '2022-09-30'
days_per_batch = 10

In [4]:
from importlib import reload
from itertools import repeat
from date_utils import partition_date_range
import pandas as pd

In [5]:
import functions as fu
reload(fu)

/app/infra/data/jupyterhub/images/singleuser_spark/main.binary.runfiles/vendor_python_geopandas/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


<module 'functions' from '/home/maximilian.hofmann/tools/brand_science_pricing/data/functions.py'>

In [6]:
from multiprocessing.dummy import Pool

In [7]:
with open('/home/maximilian.hofmann/tools/brand_science_pricing/data/queries/sql_ofo_scrape_data_new.sql', 'r') as f:
    sql_template = f.read()

In [8]:
project_name = 'css-operations'
stag_dataset_name = 'brand_science_stag'
stag_table_name = 'pricing_ofo_scrape_data_stag'
full_stag_table_name = '.'.join([project_name, stag_dataset_name, stag_table_name])

In [9]:
date_list = partition_date_range(pd.to_datetime(begin_date), pd.to_datetime(end_date), days_per_batch)
date_list

[['2022-06-01', '2022-06-15'],
 ['2022-06-16', '2022-06-30'],
 ['2022-07-01', '2022-07-15'],
 ['2022-07-16', '2022-07-30'],
 ['2022-07-31', '2022-08-14'],
 ['2022-08-15', '2022-08-29'],
 ['2022-08-30', '2022-09-13'],
 ['2022-09-14', '2022-09-28'],
 ['2022-09-29', '2022-09-30']]

In [10]:
clustering_fields=['country_code','month']

In [15]:
res = []
with Pool(5) as p:
    r = p.starmap(fu.process_ofo_data_date_range, zip(repeat(sql_template), date_list, repeat(collection), repeat(service_slug), repeat(full_stag_table_name), repeat(clustering_fields)))
    res.append(r)

Start data cleaning...
Start data cleaning...Start data cleaning...

Start data cleaning...
Data cleaning finished!Data cleaning finished!

Data cleaning finished!
Data cleaning finished!
Data cleaning finished!Data cleaning finished!
Start data cleaning...
Data cleaning finished!
Data cleaning finished!
Start data cleaning...
Data cleaning finished!


In [16]:
prod_dataset_name = 'brand_science'
prod_table_name = 'pricing_ofo_scrape_data'
full_prod_table_name = '.'.join([project_name, prod_dataset_name, prod_table_name])

In [17]:
create_tbl = fu.create_tbl(full_stag_table_name, full_prod_table_name)

In [18]:
upsert_tbl = fu.upsert_tbl_ofo_scrape_data(full_stag_table_name, full_prod_table_name)